In [1]:
import import_ipynb
import IntimeLibrary as il
import FlieLibrary as fl
import pandas as pd
import numpy as np

importing Jupyter notebook from IntimeLibrary.ipynb
importing Jupyter notebook from FlieLibrary.ipynb


In [2]:
print(fl.flight_from_cju.head(10))
print(fl.flight_to_gmp.head(10))
print(fl.flight_from_gmp.head(10))
print(fl.flight_to_cju.head(10))

         날짜     항공사  현황  time weekend DelayRate
0  20181001  아시아나항공  출발     6     Mon         1
1  20181001  아시아나항공  출발     6     Mon         1
2  20181001    대한항공  출발     7     Mon         1
3  20181001     진에어  출발     7     Mon         1
4  20181001    제주항공  출발     7     Mon         1
5  20181001  아시아나항공  출발     7     Mon         1
6  20181001   티웨이항공  출발     7     Mon         1
7  20181001    대한항공  출발     8     Mon         1
8  20181001     진에어  출발     8     Mon         1
9  20181001   이스타항공  지연     8     Mon         1
         날짜     항공사  현황  time weekend DelayRate
0  20181001  아시아나항공  도착     7     Mon         0
1  20181001  아시아나항공  도착     7     Mon         0
2  20181001     진에어  도착     8     Mon         1
3  20181001    대한항공  도착     8     Mon         0
4  20181001    제주항공  도착     8     Mon         0
5  20181001  아시아나항공  도착     9     Mon         0
6  20181001   티웨이항공  도착     9     Mon         0
7  20181001    대한항공  도착     9     Mon         0
8  20181001   이스타항공  도착     9     Mon   

In [3]:
fl.flight_from_cju = pd.get_dummies(fl.flight_from_cju, columns=["항공사", "현황", "weekend"])
fl.flight_to_gmp = pd.get_dummies(fl.flight_to_gmp, columns=["항공사", "현황", "weekend"])
fl.flight_from_gmp = pd.get_dummies(fl.flight_from_gmp, columns=["항공사", "현황", "weekend"])
fl.flight_to_cju = pd.get_dummies(fl.flight_to_cju, columns=["항공사", "현황", "weekend"])

In [4]:
fl.flight_from_cju

,날짜,time,DelayRate,항공사_대한항공,항공사_아시아나항공,항공사_에어부산,항공사_에어필립,항공사_이스타항공,항공사_제주항공,항공사_진에어,...,현황_출발,현황_취소,현황_회항,weekend_FRI,weekend_Mon,weekend_SAT,weekend_SUN,weekend_THU,weekend_TUE,weekend_WED
0,20181001,6,1,0,1,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
1,20181001,6,1,0,1,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
2,20181001,7,1,1,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
3,20181001,7,1,0,0,0,0,0,0,1,...,1,0,0,0,1,0,0,0,0,0
4,20181001,7,1,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44092,20191001,21,2,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
44093,20191001,21,1,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
44094,20191001,21,2,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
44095,20191001,21,2,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [5]:
fl.flight_to_gmp

,날짜,time,DelayRate,항공사_대한항공,항공사_아시아나항공,항공사_에어부산,항공사_에어필립,항공사_이스타항공,항공사_제주항공,항공사_진에어,...,현황_지연,현황_취소,현황_회항,weekend_FRI,weekend_Mon,weekend_SAT,weekend_SUN,weekend_THU,weekend_TUE,weekend_WED
0,20181001,7,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,20181001,7,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,20181001,8,1,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3,20181001,8,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,20181001,8,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44033,20191001,22,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
44034,20191001,22,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
44035,20191001,22,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
44036,20191001,22,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [6]:
fl.flight_from_gmp

,날짜,time,DelayRate,항공사_대한항공,항공사_아시아나항공,항공사_에어부산,항공사_에어필립,항공사_이스타항공,항공사_제주항공,항공사_진에어,...,현황_출발,현황_취소,현황_회항,weekend_FRI,weekend_Mon,weekend_SAT,weekend_SUN,weekend_THU,weekend_TUE,weekend_WED
0,20181001,6,1,0,0,0,0,0,0,1,...,1,0,0,0,1,0,0,0,0,0
1,20181001,6,1,0,1,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
2,20181001,6,1,0,0,0,0,0,0,1,...,1,0,0,0,1,0,0,0,0,0
3,20181001,6,1,0,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0
4,20181001,6,1,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44529,20191001,20,1,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
44530,20191001,20,2,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
44531,20191001,20,2,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
44532,20191001,21,1,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [7]:
fl.flight_to_cju

,날짜,time,DelayRate,항공사_대한항공,항공사_아시아나항공,항공사_에어부산,항공사_에어필립,항공사_이스타항공,항공사_제주항공,항공사_진에어,...,현황_지연,현황_취소,현황_회항,weekend_FRI,weekend_Mon,weekend_SAT,weekend_SUN,weekend_THU,weekend_TUE,weekend_WED
0,20181001,7,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,20181001,7,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,20181001,7,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
3,20181001,7,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
4,20181001,7,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44517,20191001,21,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
44518,20191001,21,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
44519,20191001,22,2,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
44520,20191001,22,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
